<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/Coil3D_Cross_Transform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [136]:
import numpy as np
import jax.numpy as jnp
import jax
from plotly.subplots import make_subplots

In [167]:


def curve(t):
    return jnp.array([jnp.cos(t), jnp.sin(t), t/(2*np.pi)])

curve_jac=jax.jacobian(curve)


In [168]:
def surf(t,theta):

    r=0.2
    vl = curve(t)
    v_jac = curve_jac(t)
    v_jac=v_jac/jnp.linalg.norm(v_jac)

    n1=jnp.cross(v_jac,jnp.array([0.,0.,1.]))
    n1=n1/jnp.linalg.norm(n1)
    n2=jnp.cross(n1,v_jac)
    n2=n2/jnp.linalg.norm(n2)

    rot = jnp.array([[jnp.cos(theta), -jnp.sin(theta)],[jnp.sin(theta),jnp.cos(theta)]])
    vx = jnp.array([r,0])
    vrot = jnp.matmul(rot, vx)
    vsurf = jnp.matmul(jnp.stack([n1,n2], axis=1),vrot)

    return vl+vsurf

In [169]:
surf_v = jnp.vectorize(surf, signature='(),()->(3)')

In [170]:
t=jnp.linspace(0,4*np.pi,120)
theta=jnp.linspace(0,2*np.pi,16)
s=surf_v(t[None,:],theta[:,None])

In [171]:
fig = make_subplots(rows=1,cols=1,specs=[[{'type': 'surface'}]])
for i in range(theta.size):
    x,y,z = s[i].T
    fig.add_scatter3d(x=x,y=y,z=z,mode='lines', line_color='white')

for i in range(t.size):
    x,y,z = s[:,i].T
    fig.add_scatter3d(x=x,y=y,z=z,mode='lines', line_color='white')

fig.update_layout(width=800,template='plotly_dark',showlegend=False)

In [ ]:
xn.shape